In [1]:
import pandas as pd

from df.keiba_data_frame import KeibaDataFrame
from df.result_frame import ResultFrame
from env.reward import Reward

# Train Data 読み込み
# ※ 既に AI 側で filter/data 整備は実施済みのため再 type 明確化と、特定列を Drop のみ行っている。
kdf = KeibaDataFrame.from_data("./../data/train/2004_binary_without_g.csv")
kdf.setting_types()

# Reward 準備
reward = Reward.initialize("./../data/train/2004_result.csv")

In [ ]:
import numpy as np
from tensorflow import keras

new_model = keras.models.load_model("./../model/binary_model_without_g")
new_model.summary()

preds = []

i = 0
for v in list(kdf.groups()):
    v.reset_index(inplace=True, drop=True)
    #obs = v.reindex(range(20), fill_value=-1).values.reshape(1, 1, 20, 51).tolist()
    obs = v[["odds", "pred"]].values.reshape(1, 1, 3, 2).tolist()
    #print(obs)
    q_values = new_model.predict(obs)
    #print(q_values)
    action = np.argmax(q_values[0])
    #print(action)
    pred_df = pd.DataFrame(
        {
            "track_id": v["track_id"],
            "date_num": v["date_num"],
            "round": v["round"],
            "number": v["number"],
            "pred": v["pred"],
            "action": action,
        }
    )
    preds.append(pred_df)
    # i += 1
    # if i > 100:
    #     break

preds_df = pd.concat(preds)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 6)                 0         
                                                                 
 dense (Dense)               (None, 128)               896       
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 4)                 516       
                                                                 
Total params: 34,436
Trainable params: 34,436
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 0s 22ms/step


In [ ]:
mikounyu_count = 0
kounyu_count = 0
atari_count = 0
hazure_count = 0
reward_total = 0

action_0_count = 0
action_1_count = 0
action_2_count = 0

df = preds_df.sort_values(
    ["date_num", "track_id", "round"], ascending=[True, True, True]
)

for _, v in df.groupby(["date_num", "track_id", "round"], sort=False):
    action = int(v.iloc[0]["action"])

    if action == 0:
        action_0_count += 1
    elif action == 1:
        action_1_count += 1
    elif action == 2:
        action_2_count += 1

    if action == 3:
        reward_total += 0
        mikounyu_count += 1
    else:
        kounyu_count += 1
        result = reward.calc(v, action)
        if result > 0:
            atari_count += 1
        else:
            hazure_count += 1

        reward_total += result

print(f"RANK_ONE_HORSE 回数: {action_0_count}")
print(f"RANK_TWO_HORSE 回数: {action_1_count}")
print(f"RANK_THREE_HORSE 回数: {action_2_count}")
print(f"未購入回数: {mikounyu_count}")
print(f"当たり馬券: {atari_count}")
print(f"はずれ馬券: {hazure_count}")
print(f"的中率: {(atari_count / (atari_count + hazure_count))}")
print(f"儲け: {reward_total}")